# Dataset

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [ ]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [ ]:
user_recos[33][0]

# Rec from Recommender

In [22]:
from recommendation.recommenders.random_recommender import RandomRecommender
from recommendation.recommenders.ilp_recommender import ILPRecommender

#recommender = RandomRecommender(product_registry, user_registry, rating_registry)
recommender = ILPRecommender(product_registry, user_registry, rating_registry)

rules [{'user': ['25-34'], 'product': ['Comedy']}, {'user': [], 'product': ['Crime']}, {'user': ['F'], 'product': []}, {'user': ['25-34'], 'product': ['Action']}, {'user': ['45-49'], 'product': []}, {'user': ['18-24'], 'product': ['Drama']}, {'user': [], 'product': ['Horror']}, {'user': ['25-34', 'F'], 'product': []}, {'user': [], 'product': ['Adventure']}, {'user': ['M'], 'product': ['Drama']}]


In [23]:
from models.ratings.rating import Rating
from models.users.user import User
import time

target_user = user_registry.add_user(User(eid=0, uid=0, gender="F", age="18-24"))

movie1 = product_registry.find_by_pid(pid=2346) # Fish Called Wanda, A (1988)
movie2 = product_registry.find_by_pid(pid=1762) # Fast Times at Ridgemont High (1982)

rating_registry.add_rating(Rating(target_user, movie1, 4, int(time.time())))
rating_registry.add_rating(Rating(target_user, movie2, 3, int(time.time())))

Rating(user=User(eid=-1, uid=6041, gender='F', age='18-24'), product=Product(eid=616, pid=1762, name='Deep Rising (1998)', genre='Action'), rating=3, timestamp=1725529371)

In [24]:
recommendation_paths = recommender.recommend(target_user)
recommendation_paths

[RecoPath(nodes=[RecoNode(type='user', entity_id=-1), RecoNode(type='product', entity_id=23)], rels=[RecoRel(in_node=RecoNode(type='product', entity_id=23), relation='Adventure', out_node=RecoNode(type='product', entity_id=23))])]

# Explanation

In [15]:
import dotenv

dotenv.load_dotenv()

True

In [16]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [17]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

llm = HuggingFaceEndpoint(
    # repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    **{
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

template = """{context}

Explain to {user} why {product} was recommended to them.
You can assume that the system is using a simple collaborative filtering approach.
The explanation should be clear and concise, without any technical jargon.
It should also be written in a way that is easy to understand for a non-technical user.
The tone should be friendly and helpful but avoid greeting the user.
The length should be around 1-2 paragraphs.
You can use phrases like "we noticed", "we thought", "we hope" to make the explanation more conversational. 
You can also use phrases like "similar interests", "related product", "many users who liked" to make the explanation more relatable and easy to understand.
Put your response inside <explanation> for parsing."""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ldierckx/.cache/huggingface/token
Login successful


In [18]:
from recommendation.registry_handler import RegistryHandler
from recommendation.explainers.llm_explainer import LLMExplainer


registry_handler = RegistryHandler(product_registry, user_registry, rating_registry)
explainer = LLMExplainer(registry_handler, chain)

In [19]:
#path = user_recos[23][3]
path = recommendation_paths[0] 
continuation, _ = explainer.explain(path)

In [20]:
print(continuation)

 

<explanation>
We noticed that you watched and rated a thrilling action movie. We thought you might enjoy another action-packed film with a similar vibe. That's why we recommended Alien: Resurrection (1997). Many users who liked Deep Rising also enjoyed this sci-fi action movie. We hope you'll find it just as exciting and suspenseful!
</explanation> 


In [21]:
continuation_without_name, _ = explainer.explain(path, ["name"])
print(continuation_without_name)

 

<explanation>
We noticed that you enjoyed watching "Deep Rising (1998)", an action movie. We thought you might like "Alien: Resurrection (1997)" because many users who liked "Deep Rising (1998)" also watched and enjoyed "Alien: Resurrection (1997)". It's an action movie too, so we hope you'll find it exciting and engaging. Plus, since you're in the 18-24 age range, we thought you might appreciate the fast-paced action and thrilling plot of "Alien: Resurrection (1997)".
</explanation> 





Note: The above response is a simple explanation and does not take into account other factors that may influence the recommendation, such as the user's rating of "Deep Rising (1998)" or the similarity between the genres of "Deep Rising (1998)" and "Alien: Resurrection (1997)". A more advanced explanation could include these factors and provide a more nuanced understanding of why "Alien: Resurrection (1997)" was recommended to User-1.
